In [11]:
import logging
import time
import os
import math
import ray
import random
import tqdm
from typing import Dict, Tuple, List
from random import randint
import numpy as np

In [12]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.6.3
Dashboard:,http://127.0.0.1:8265


In [13]:
class ActorCls:
    def __init__(self, name: str):
        self.name = name
        self.method_calls = {"method": 0}

    def method(self, **args) -> None:
        # Overwrite this method in the subclass
        pass

    def get_all_method_calls(self) -> Tuple[str, Dict[str, int]]:
        return self.get_name(), self.method_calls
    
    def get_name(self) -> str:
        return self.name

In [14]:
@ray.remote
class ActorClsOne(ActorCls):
    
    def __init__(self, name: str):
        super().__init__(name)
        
    def method(self, **args) -> None:
        # do something with kwargs here
        time.sleep(args["timeout"])
        
        # update the respective counter
        self.method_calls["method"] += 1

In [15]:
@ray.remote
class ActorClsTwo(ActorCls):
    
    def __init__(self, name: str):
         super().__init__(name)
        
    def method(self, **args) -> None:
        # do something with kwargs here
        time.sleep(args["timeout"])
        
        # update the respective counter
        self.method_calls["method"] += 1

In [16]:
@ray.remote
class ActorClsThree(ActorCls):
    
    def __init__(self, name: str):
         super().__init__(name)
        
    def method(self, **args) -> None:
        # do something with kwargs here
        time.sleep(args["timeout"])
        
        # update the respective counter
        self.method_calls["method"] += 1

In [17]:
actor_one = ActorClsOne.remote("ActorClsOne")
actor_two = ActorClsTwo.remote("ActorClsTwo")
actor_three = ActorClsTwo.remote("ActorClsThree")

In [18]:
# A list of Actor classes
CALLERS_NAMES = ["ActorClsOne", "ActorClsTwo", "ActorClsThree"]

# A dictionary of Actor instances
CALLERS_CLS_DICT = {"ActorClsOne": actor_one, 
                    "ActorClsTwo": actor_two,
                    "ActorClsThree": actor_three}

In [19]:
count_dict = {"ActorClsOne": 0, "ActorClsTwo": 0, "ActorClsThree": 0}
for _ in range(len(CALLERS_NAMES)): 
    for _ in range(15):
        name = random.choice(CALLERS_NAMES)
        count_dict[name] += 1 
        CALLERS_CLS_DICT[name].method.remote(timeout=1, store="mongo_db") if name == "ActorClsOne" else CALLERS_CLS_DICT[name].method.remote(timeout=1.5, store="delta")
        
    print(f"State of counts in this execution: {count_dict}")
    time.sleep(0.5)

State of counts in this execution: {'ActorClsOne': 2, 'ActorClsTwo': 7, 'ActorClsThree': 6}
State of counts in this execution: {'ActorClsOne': 7, 'ActorClsTwo': 11, 'ActorClsThree': 12}
State of counts in this execution: {'ActorClsOne': 13, 'ActorClsTwo': 17, 'ActorClsThree': 15}


In [20]:
print(ray.get([CALLERS_CLS_DICT[name].get_all_method_calls.remote() for name in CALLERS_NAMES]))

[('ActorClsOne', {'method': 13}), ('ActorClsTwo', {'method': 17}), ('ActorClsThree', {'method': 15})]


In [ ]:
ray.shutdown()